# [Advent of Code 2024](https://adventofcode.com/2024 )

## General functions

In [1]:
import re
import itertools
import math
from operator import itemgetter

# import matplotlib.pyplot as plt
# import numpy as np

import requests


In [2]:
def Input(day, year=2024):
    directory = '{}'.format(year)
    filename = directory + '/input{}.txt'.format(day)
    return open(filename)

def Testdata(day, year=2024):
    directory = '{}'.format(year)
    filename = directory + '/testdata{}.txt'.format(day)
    return open(filename)

def mapt(fn, *args):
    """map(fn, *args) and return the result as a tuple."""
    return tuple(map(fn, *args))

def parse(day, parser=str, sep='\n', output='tuple') -> tuple:
    """Split the day's input file into entries separated by `sep`, and apply `parser` to each."""
    entries = open(f'2024/input{day}.txt').read().rstrip().split(sep)
    return mapt(parser, entries)

In [3]:
def findroots(a, b, c):
    dis = b * b - 4 * a * c 
    sqrt_val = math.sqrt(abs(dis)) 
    if dis > 0:
        return (-b + sqrt_val)/(2 * a), (-b - sqrt_val)/(2 * a)
    elif dis == 0:
        return -b / (2 * a)
    else:
        return - b / (2 * a), + i, sqrt_val,  - b / (2 * a), - i, sqrt_val

## [Day 1: Historian Hysteria](https://adventofcode.com/2024/day/1)

In [4]:
def list_diff(lst1, lst2):
    return [abs(e[0] - e[1]) for e in zip(sorted(lst1), sorted(lst2))]

def parselst(str):
    str = list(zip(*[e.split() for e in str.strip().split("\n")]))
    lst1, lst2 = str
    lst1 = [int(e) for e in lst1]
    lst2 = [int(e) for e in lst2]
    return lst1, lst2

In [5]:
teststr = "3   4\n4   3\n2   5\n1   3\n3   9\n3   3"
assert sum(list_diff(*parselst(teststr)))==11

In [6]:
inp = Input('01').read()
sum(list_diff(*parselst(inp)))

1110981

In [7]:
def similarity(lst1, lst2):
    total = 0
    for e in lst1:
        total += e * lst2.count(e)
    return total

In [8]:
assert similarity(*parselst(teststr)) == 31

In [9]:
similarity(*parselst(inp))

24869388

## [Day 2: Red-Nosed Reports](https://adventofcode.com/2024/day/2)

In [10]:
def parseline(line,d=False):
    t = [int(e) for e in line.split(' ')]
    return [e - t[i -1] for i, e in enumerate(t)]

def issafe(e):
    t1 = [True if (e < 0 and e >= -3) else False for e in parseline(e)][1:]
    t2 = [True if (e > 0 and e <= 3) else False for e in parseline(e)][1:]
    return False in t1 and False in t2

def allsafe(text):
    text = text.strip().split('\n')
    return [0 if issafe(e) == True else 1 for e in text]

In [11]:
teststr = "7 6 4 2 1\n1 2 7 8 9\n9 7 6 2 1\n1 3 2 4 5\n8 6 4 4 1\n1 3 6 7 9"
sum(allsafe(teststr))

2

In [12]:
inp = Input('02').read()
sum(allsafe(inp))


472

In [13]:
def parseline(line,d=False):
    return [int(e) for e in line.split(' ')]

def issafe(e, d=False):
    o, out = [e], []
    if d == True:
        for i in range(len(e)):
            o.append(e[:i] + e[i+1:])
    for l in o:
        t = [e - l[i -1] for i, e in enumerate(l)]
        t1 = [True if (e < 0 and e >= -3) else False for e in t[1:]]
        t2 = [True if (e > 0 and e <= 3) else False for e in t[1:]]
        if False not in t1 or False not in t2:
            return True
    return False

def allsafe(text, d=False):
    text = text.strip().split('\n')
    ct = 0
    for line in text:
        line = parseline(line)
        if issafe(line, d):
            ct += 1
    return ct

In [14]:
assert allsafe(teststr, d=True) == 4

Still works for part 1?

In [15]:
assert allsafe(inp) == 472

In [16]:
allsafe(inp, d=True)

520

## [Day 3: Mull It Over](https://adventofcode.com/2024/day/3)

In [17]:
def get_ops(str):
    return re.findall(r'mul\((\d{1,3}),(\d{1,3})\)', str)

def multiplications(str):
    return [int(e[0]) * int(e[1]) for e in get_ops(str)]

In [18]:
teststr = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"
assert sum(multiplications(teststr))==161

In [19]:
get_ops(teststr)

[('2', '4'), ('5', '5'), ('11', '8'), ('8', '5')]

In [20]:
inp = Input('03').read()
sum(multiplications(inp))

189600467

In [21]:
def get_ops(str):
    return re.findall(r'mul\((\d{1,3}),(\d{1,3})\)|(do[n\'t]*)', str)
def mults(str):
    res = 0
    multi = 1
    m = get_ops(str)
    for e in m:
        if e[2] == "don't":
            multi = 0
        elif e[2] == "do":
            multi = 1
        else:
            res += int(e[0]) * int(e[1]) * multi
    return res

In [22]:
teststr = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"
assert mults(teststr) ==48

In [23]:
mults(inp)

107069718

In [24]:
get_ops(teststr)

[('2', '4', ''),
 ('', '', "don't"),
 ('5', '5', ''),
 ('11', '8', ''),
 ('', '', 'do'),
 ('8', '5', '')]

## [Day 4: Ceres Search](https://adventofcode.com/2024/day/4)

In [25]:
def word_search(m, w):
    ct = 0
    lst = []
    for i, l in enumerate(m):
        for j, c in enumerate(l):
            if c == w[0]:
                for dx in (-1, 0, 1):
                    for dy in (-1, 0, 1):
                        try:
                            wd = [m[i + dx * k][j + dy * k] for k in range(len(w)) if i + dx * k >= 0 and j + dy * k >= 0]
                        except:
                            break
                        if "".join(wd) == w:
                            ct += 1
                            lst.append((i, j))
    return ct

In [26]:
teststr= "MMMSXXMASM\nMSAMXMSMSA\nAMXSXMAAMM\nMSAMASMSMX\nXMASAMXAMM\nXXAMMXXAMA\nSMSMSASXSS\nSAXAMASAAA\nMAMMMXMMMM\nMXMXAXMASX"
teststr = [list(e) for e in teststr.split('\n')]
assert word_search(teststr, "XMAS") ==18

In [27]:
inp = [list(e) for e in Input('04').readlines()]
word_search(inp, "XMAS")

2685

In [28]:
def word_search2(m, w):
    ct=0
    lst = []
    for i, l in enumerate(m):
        for j, c in enumerate(l):
            sct = 0
            if c == w[int(len(w) / 2)]:                
                for dx in (-1, 1):
                    for dy in (-1, 1):
                        try:
                            wd = [m[i + dx * k][j + dy * k] for k in range(-int(len(w) / 2), int(len(w) / 2) + 1) if i + dx * k >= 0 and j + dy * k >= 0]
                        except:
                            break
                        if "".join(wd) == w:
                            sct += 1
                            lst.append((i, j))
            if sct > 1:
                ct += 1
    return ct
    


In [29]:
assert word_search2(teststr, "MAS") ==9

In [30]:
word_search2(inp, "MAS")

2048